In [53]:
import numpy as np
import jax.numpy as jnp
from jax import lax
import jax

from jax import config
config.update("jax_enable_x64", True)

# note to self: wrap angles before passing to kepler solver? 
# add deep space logic and potentially error checks later

# ==============================================================================
# SGP4 CONSTANTS (WGS-72 / Appendix B)
# ==============================================================================

# Gravitational constants (WGS-72)
J2 = 1.082616e-3 # Unnormalised zonal harmonic coefficients
J3 = -0.253881e-5
J4 = -1.65597e-6
GM = 3.986008e5 # Earth gravitational parameter km^3/s^2
aE = 6378.135   # Earth equatorial radius in km 
ke = 60.0 / jnp.sqrt(aE**3 / GM)  # sqrt(GM) in units (Earth Radii)^1.5 / min)

# Derived constants (note: in normalised units as implied by paper i.e. aE = 1) 
k2 = 0.5 * J2 # (Earth Radii)^2 
A30 = -J3 # Normalized
k4 = -3/8 * J4 # Normalized

# Density constants
q0_val = 120.0  # km
s_val = 78.0    # km

#@jax.jit (commented this out to compare timings of jitted vs non-jitted)
def sgp4(n0, e0, i0, w0, Omega0, M0, t0, Bstar, tsince):
    """
    SGP4 propagation algorithm.
    
    Inputs:
      n0      : Mean motion (revs/day)
      e0      : Eccentricity
      i0      : Inclination (degrees)
      w0      : Argument of perigee (degrees)
      Omega0  : Right ascension of ascending node (degrees)
      M0      : Mean anomaly (degrees)
      t0      : Epoch time (minutes from some reference, usually not needed for relative calc)
      Bstar   : Drag coefficient (1/Earth Radii)
      tsince  : Time since epoch (minutes)
      
    Returns:
      r       : Position vector [x, y, z] in km
      v       : Velocity vector [vx, vy, vz] in km/s
      (changed this to output concatenated array of r and v to make timing easier)
    """
    
    # --------------------------------------------------------------------------
    # A. INITIALIZATION
    # --------------------------------------------------------------------------
    
    # note to self: paper implicity uses units of Earth Radii for distance, minutes for time and radians 
    # for all internal calculations. (Output converted to km and km/s at end).

    # Convert inputs to radians
    i0 = jnp.radians(i0)
    w0 = jnp.radians(w0)
    Omega0 = jnp.radians(Omega0)
    M0 = jnp.radians(M0)
    
    # Convert Mean Motion from revs/day to rad/min
    n0_rad_min = n0 * (2 * jnp.pi) / 1440.0

    # Recover Brouwer mean motion from Kozai mean motion
    a1 = (ke / n0_rad_min) ** (2 / 3)
    
    # (Precompute trig) 
    cos_i0 = jnp.cos(i0)
    sin_i0 = jnp.sin(i0)
    theta = cos_i0
    theta2 = theta ** 2
    theta4 = theta ** 4
    
    delta1 = (1.5 * k2 * (3 * theta2 - 1)) / ((1 - e0 ** 2) ** 1.5 * a1 ** 2)
    a2 = a1 * (1 - delta1 / 3 - delta1 ** 2 - 134 / 81 * delta1 ** 3) 
    delta0 = (1.5 * k2 * (3 * theta2 - 1)) / ((1 - e0 ** 2) ** 1.5 * a2 ** 2)

    n0_brouwer = n0_rad_min / (1 + delta0)
    a0_brouwer = (ke / n0_brouwer) ** (2 / 3)

    # 1. Initialization for Secular Effects of Atmospheric Drag
    # ----------------------------------------------------------

    # Calculate epoch perigee height (in km)
    rp = (a0_brouwer * (1 - e0) - 1.0) * aE 
    
    # Logic for 's' parameter (in Earth Radii) based on perigee height
    s = lax.cond(
        rp >= 156,
        lambda rp: (s_val / aE) + 1.0,
        lambda rp: lax.cond(
            rp >= 98,
            lambda rp: (rp - s_val) / aE + 1.0,
            lambda rp: (20.0 / aE) + 1.0,
            rp
        ), 
        rp
    )
    
    # q0 parameter in Earth Radii
    q0 = (q0_val / aE) + 1.0

    xi = 1.0 / (a0_brouwer - s)
    beta0 = jnp.sqrt(1 - e0 ** 2)
    eta = a0_brouwer * e0 * xi
    
    # C2 Calculation
    term_c2_1 = a0_brouwer * (1 + 1.5 * eta ** 2 + 4 * e0 * eta + e0 * eta ** 3)
    term_c2_2 = (1.5 * k2 * xi / (1 - eta ** 2)) * (-0.5 + 1.5 * theta2) * (8 + 24 * eta ** 2 + 3 * eta ** 4)
    C2 = (q0 - s) ** 4 * xi ** 4 * n0_brouwer * (1 - eta ** 2) ** (-3.5) * (term_c2_1 + term_c2_2)
    
    C1 = Bstar * C2
    #CHECK whether this divide by zero check is nececessary / correct
    # C3 = jnp.where(e0 > 1e-4, 
    #                (q0 - s) ** 4 * xi ** 5 * A30 * n0_brouwer * sin_i0 / (k2 * e0), 
    #                0.0) # Avoid divide by zero for circular orbits
    C3 = (q0 - s) ** 4 * xi ** 5 * A30 * n0_brouwer * sin_i0 / (k2 * e0)

    # C4 Calculation
    term_c4_1 = 2 * eta * (1 + e0 * eta) + 0.5 * e0 + 0.5 * eta ** 3
    term_c4_2 = (2 * k2 * xi / (a0_brouwer * (1 - eta ** 2))) * \
                (3 * (1 - 3 * theta2) * (1 + 1.5 * eta ** 2 - 2 * e0 * eta - 0.5 * e0 * eta ** 3) + 
                 0.75 * (1 - theta2) * (2 * eta ** 2 - e0 * eta - e0 * eta ** 3) * jnp.cos(2 * w0))
    
    C4 = 2 * n0_brouwer * (q0 - s) ** 4 * xi ** 4 * a0_brouwer * beta0 ** 2 * \
         (1 - eta ** 2) ** (-3.5) * (term_c4_1 - term_c4_2)
         
    C5 = 2 * (q0 - s) ** 4 * xi ** 4 * a0_brouwer * beta0 ** 2 * \
         (1 - eta ** 2) ** (-3.5) * (1 + 2.75 * eta * (eta + e0) + e0 * eta ** 3)
         
    D2 = 4 * a0_brouwer * xi * C1 ** 2
    D3 = (4 / 3) * a0_brouwer * xi ** 2 * (17 * a0_brouwer + s) * C1 ** 3
    D4 = (2 / 3) * a0_brouwer ** 2 * xi ** 3 * (221 * a0_brouwer + 31 * s) * C1 ** 4

    # 2. Initialization for Secular Effects of Earth Zonal Harmonics
    # --------------------------------------------------------------

    Mdot = n0_brouwer * (3 * k2 * (-1 + 3 * theta2) / (2 * a0_brouwer ** 2 * beta0 ** 3) + 
                         3 * k2 ** 2 * (13 - 78 * theta2 + 137 * theta4) / (16 * a0_brouwer ** 4 * beta0 ** 7))
    
    wdot = n0_brouwer * (-3 * k2 * (1 - 5 * theta2) / (2 * a0_brouwer ** 2 * beta0 ** 4) + 
                         3 * k2 ** 2 * (7 - 114 * theta2 + 395 * theta4) / (16 * a0_brouwer ** 4 * beta0 ** 8) + 
                         5 * k4 * (3 - 36 * theta2 + 49 * theta4) / (4 * a0_brouwer ** 4 * beta0 ** 8))
                         
    Omegadot = n0_brouwer * (-3 * k2 * theta / (a0_brouwer ** 2 * beta0 ** 4) + 
                             3 * k2 ** 2 * (4 * theta - 19 * theta ** 3) / (2 * a0_brouwer ** 4 * beta0 ** 8) + 
                             5 * k4 * theta * (3 - 7 * theta2) / (2 * a0_brouwer ** 4 * beta0 ** 8))

    # 3. Initialization for Deep Space (Logic Hook)
    # Check for Deep Space (Period >= 225 minutes)
    period_min = 2 * jnp.pi / n0_rad_min
    is_deep_space = period_min >= 225.0
    
    # [ADDITION] Deep Space Initialization placeholder
    # In a full implementation, this calculates the Z-terms from Appendix A.
    # For this transcription, we initialize rates to zero unless detailed logic is added.
    # note to self: if deep space then not zero, else zero
    dot_M_LS = 0.0
    dot_w_LS = 0.0
    dot_Omega_LS = 0.0
    dot_e_LS = 0.0
    dot_i_LS = 0.0
    
    # If deep space logic were fully expanded here, we would populate the above variables
    # using the Z coefficients (Z1..Z33) and Lunar/Solar constants.

    # --------------------------------------------------------------------------
    # B. UPDATE
    # --------------------------------------------------------------------------
    
    # 1. Secular Update for Earth Zonal Gravity and Partial Atmospheric Drag Effects
    # ------------------------------------------------------------------------------

    MDF = M0 + (n0_brouwer + Mdot) * tsince
    wDF = w0 + wdot * tsince
    OmegaDF = Omega0 + Omegadot * tsince
    
    # Check for perigee height < 220 km for drag terms (will need to also add deep space check later)
    is_high_perigee = rp >= 220.0
    
    deltaw = lax.cond(
        is_high_perigee, 
        lambda _: Bstar * C3 * jnp.cos(w0) * tsince, 
        lambda _: 0.0,
        operand=None   
    )
    
    deltaM = lax.cond(
        is_high_perigee,
        lambda _: -2/3 * (q0 - s) ** 4 * Bstar * xi ** 4 / (e0 * eta) *
                  ((1 + eta * jnp.cos(MDF)) ** 3 - (1 + eta * jnp.cos(M0)) ** 3),
        lambda _: 0.0,
        operand=None
    )

    M_secular = MDF + deltaw + deltaM
    w_secular = wDF - deltaw - deltaM
    Omega_secular = OmegaDF - 21/2 * (n0_brouwer * k2 * theta / (a0_brouwer ** 2 * beta0 ** 2)) * C1 * tsince ** 2

    # 2. Secular Updates for Lunar and Solar Gravity (Deep Space)
    # --------------------------------------------------------------

    # [ADDITION] Apply the secular rates derived from Deep Space analysis
    # Note: These are 0.0 if period < 225 min
    M_secular += dot_M_LS * tsince
    w_secular += dot_w_LS * tsince
    Omega_secular += dot_Omega_LS * tsince
    e_secular = e0 + dot_e_LS * tsince
    i_secular = i0 + dot_i_LS * tsince

    # 3. Secular Updates for Resonance Effects of Earth Gravity
    # --------------------------------------------------------------
    # Add this later

    # 4. Secular Update for Remaining Atmospheric Drag Effects
    # --------------------------------------------------------------
    
    # Define time powers
    t2 = tsince ** 2
    t3 = tsince ** 3
    t4 = tsince ** 4

    # CHECK which terms to cut off for this condition? paper says 'linear' term which is ambiguous

    # Branch 1: High Perigee (>= 220km)
    def e_high(_):
        e = e_secular - Bstar * C4 * tsince - Bstar * C5 * (jnp.sin(M_secular) - jnp.sin(M0))
        a = (ke / n0_brouwer) ** (2/3) * (1 - C1 * tsince - D2 * t2 - D3 * t3 - D4 * t4) ** 2
        IL = M_secular + w_secular + Omega_secular + n0_brouwer * \
                 (1.5 * C1 * t2 + (D2 + 2 * C1 ** 2) * t3 + 
                  0.25 * (3 * D3 + 12 * C1 * D2 + 10 * C1 ** 3) * t4 + 
                  0.2 * (3 * D4 + 12 * C1 * D3 + 6 * D2 ** 2 + 30 * C1 ** 2 * D2 + 15 * C1 ** 4) * tsince ** 5)
        return e, a, IL
    
    # Branch 2: Low Perigee (< 220km)
    def e_low(_):
        e = e0 - Bstar * C4 * tsince
        a = (ke / n0_brouwer) ** (2/3) * (1 - C1 * tsince) ** 2
        IL = M_secular + w_secular + Omega_secular + n0_brouwer * \
                 (1.5 * C1 * t2 + (D2 + 2 * C1 ** 2) * t3 + 
                  0.25 * (3 * D3 + 12 * C1 * D2 + 10 * C1 ** 3) * t4)
        return e, a, IL
    
    # Select based on perigee condition
    e_final_sec, a_final_sec, IL = lax.cond(
        is_high_perigee, 
        e_high, 
        e_low,
        operand=None
    )

    # note to self: not sure whether to keep this? 
    # Enforce eccentricity limits (0 <= e < 1)
    #e_final_sec = jnp.clip(e_final_sec, 1e-6, 1.0 - 1e-6)
    
    # Calculate Mean Motion 'n' at time t
    n = ke / (a_final_sec ** 1.5)
    beta = jnp.sqrt(1 - e_final_sec ** 2)

    # 5. Update for Long-Period Periodic Effects of Lunar and Solar Gravity
    # ---------------------------------------------------------------------

    # add this later

    # 6. Update for Long-Period Periodic Effects of Earth Gravity
    # -----------------------------------------------------------

    # Calculate axN, ayN
    # Note: Phase 1 of long period periodics
    axN = e_final_sec * jnp.cos(w_secular)
    
    term_ill = (A30 * jnp.sin(i_secular)) / (8 * k2 * a_final_sec * beta ** 2)
    ILL = term_ill * axN * (3 + 5 * jnp.cos(i_secular)) / (1 + jnp.cos(i_secular))
    ayNL = A30 * jnp.sin(i_secular) / (4 * k2 * a_final_sec * beta ** 2)
    
    ILT = IL + ILL
    ayN = e_final_sec * jnp.sin(w_secular) + ayNL

    # 7. Update for Short-Period Periodic Effects of Earth Gravity
    # ------------------------------------------------------------
    
    # Solve Kepler's Equation for (E + w)
    # The variable U = M + w + Omega (modified) - Omega = M + w (? idk what this line is about)
    U = ILT - Omega_secular
    
    # Fixed-Point Iteration Solver for Kepler's Equation
    # We are solving for Ew = (E + w)
    # Iteration: Ew_new = Ew_old + Delta
    
    def kepler_body(i, Ew_curr):
        numerator = U - ayN * jnp.cos(Ew_curr) + axN * jnp.sin(Ew_curr) - Ew_curr
        denominator = 1 - ayN * jnp.sin(Ew_curr) - axN * jnp.cos(Ew_curr)
        return Ew_curr + numerator / denominator
    
    # Initial guess
    Ew_initial = U
    
    # Run 10 iterations
    Ew = lax.fori_loop(0, 10, kepler_body, Ew_initial)

    # sgp4 python also has a cut off if correction becomes negligibly small and a limit on step size
    # I could do this later but i think 10 iterations is fine for now
    # if I want to add cut off condition then use lax.while_loop instead

    # Calculate preliminary quantities for short-period periodics
    ecosE = axN * jnp.cos(Ew) + ayN * jnp.sin(Ew)
    esinE = axN * jnp.sin(Ew) - ayN * jnp.cos(Ew)
    
    e_osc = jnp.sqrt(ecosE ** 2 + esinE ** 2) # note to self: this is diff. to exact eq. in paper but gives same result
    #e_osc = jnp.clip(e_osc, 1e-6, 1.0 - 1e-6) # Safety clip - CHECK if needed later
    
    pL = a_final_sec * (1 - e_osc ** 2)
    r = a_final_sec * (1 - ecosE)
    
    rdot = ke * jnp.sqrt(a_final_sec) / r * esinE
    rfdot = ke * jnp.sqrt(pL) / r
    
    # Arguments of Latitude (u)
    # Note: e in the denominator term comes from the secular/long-period mix (e_osc logic)
    # The paper uses 'e' in the denominator term `1 + sqrt(1-e^2)`. 
    # Standard implementations use the osculating e here.
    cosu = (a_final_sec / r) * (jnp.cos(Ew) - axN + ayN * esinE / (1 + jnp.sqrt(1 - e_osc ** 2)))
    sinu = (a_final_sec / r) * (jnp.sin(Ew) - ayN - axN * esinE / (1 + jnp.sqrt(1 - e_osc ** 2)))
    u = jnp.arctan2(sinu, cosu)

    # Short-Period Corrections
    sin2u = jnp.sin(2 * u)
    cos2u = jnp.cos(2 * u)
    
    # Corrections (Deltas)
    # Using i_secular (mean inclination) for these factors
    sin_i = jnp.sin(i_secular)
    cos_i = jnp.cos(i_secular)
    
    Deltar = (k2 / (2 * pL)) * (1 - cos_i ** 2) * cos2u
    Deltau = (-k2 / (4 * pL ** 2)) * (7 * cos_i ** 2 - 1) * sin2u
    DeltaOmega = (3 * k2 * cos_i / (2 * pL ** 2)) * sin2u
    Deltai = (3 * k2 * cos_i / (2 * pL ** 2)) * sin_i * cos2u
    Deltardot = (-k2 * n / pL) * (1 - cos_i ** 2) * sin2u
    Deltarfdot = (k2 * n / pL) * ((1 - cos_i ** 2) * cos2u - 1.5 * (1 - 3 * cos_i ** 2))

    # Osculating Elements
    rk = r * (1 - 1.5 * k2 * jnp.sqrt(1 - e_osc ** 2) / (pL ** 2) * (3 * cos_i ** 2 - 1)) + Deltar
    uk = u + Deltau
    Omegak = Omega_secular + DeltaOmega
    ik = i_secular + Deltai
    rdotk = rdot + Deltardot
    rfdotk = rfdot + Deltarfdot

    # --------------------------------------------------------------------------
    # C. VECTORS (Position and Velocity)
    # --------------------------------------------------------------------------
    
    # Orientation Vectors (M and N)
    M = jnp.array([
        -jnp.sin(Omegak) * jnp.cos(ik), 
        jnp.cos(Omegak) * jnp.cos(ik), 
        jnp.sin(ik)
    ])

    N = jnp.array([
        jnp.cos(Omegak), 
        jnp.sin(Omegak), 
        0.0
    ])

    # Precompute trig
    sin_uk = jnp.sin(uk)
    cos_uk = jnp.cos(uk)

    # U and V vectors
    U = M * sin_uk + N * cos_uk
    V = M * cos_uk - N * sin_uk

    # Position and Velocity in TEME frame (Earth Radii and Earth Radii/min)
    # converted to output unit Distance = km, Velocity = km/s in one step
    r_vec = rk * U * aE
    v_vec = (rdotk * U + rfdotk * V) * aE / 60.0

    return jnp.concatenate((r_vec, v_vec))

    # # If output as separate r_vec, v_vec desired:
    #return r_vec, v_vec

In [54]:
# Test case

from sgp4.api import Satrec
from sgp4.api import jday

# TLE for the International Space Station (ZARYA)
tle1 = '1 25544U 98067A   25316.22557474  .00019835  00000-0  36009-3 0  9990'
tle2 = '2 25544  51.6334 293.5281 0004132  61.4658 298.6746 15.49560899538121' 

# Create Satrec object
sat = Satrec.twoline2rv(tle1, tle2)

# Choose a time since epoch to propagate to (minutes)
tsince = 500.0

# Propagate using standard SGP4 implementation
e, r, v = sat.sgp4_tsince(tsince)

print(r, v)  # position (km), velocity (km/s)
print("SGP4 error code:", e)

(705.08880216296, 5760.37365185177, 3527.6041455633244) (-5.192094694881382, 3.401162766577282, -4.496405865253736)
SGP4 error code: 0


In [55]:
# get elements for input to our sgp4 function
n0_test = sat.no_kozai * 1440.0 / (2 * jnp.pi)  # revs/day
e0_test = sat.ecco
i0_test = sat.inclo * 180.0 / jnp.pi            # degrees
w0_test = sat.argpo * 180.0 / jnp.pi           # degrees
Omega0_test = sat.nodeo * 180.0 / jnp.pi
M0_test = sat.mo * 180.0 / jnp.pi               # degrees
t0_test = 0.0                                   # minutes
Bstar_test = sat.bstar                          # Earth radii^-1

# # If output as separate r_vec, v_vec:
# r_vec, v_vec = sgp4(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince)
# print(r_vec, v_vec)  # position (km), velocity (km/s)

result = sgp4(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince)
r_vec = result[:3]
v_vec = result[3:]
print(r_vec, v_vec)  # position (km), velocity (km/s)

[ 705.08880216 5760.37365185 3527.60414556] [-5.19209469  3.40116277 -4.49640587]


In [56]:
# Compare results
r = jnp.array(r)
v = jnp.array(v)
print(r_vec - r)
print(v_vec - v)

[ 6.03677108e-11 -3.81987775e-11  4.82032192e-11]
[1.42108547e-14 8.26005930e-14 4.79616347e-14]


In [57]:
# time sgp4 unjitted
%timeit sgp4(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince).block_until_ready()

68.7 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
# time sgp4 jitted
jaxsgp4 = jax.jit(sgp4)
%timeit jaxsgp4(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince).block_until_ready()

9.05 μs ± 150 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [51]:
# (Verify sgp4 is using the fast C++ implementation rather than the slow python)
from sgp4.api import accelerated
print(accelerated)

True


In [36]:
# time sgp4 python package C++ implementation
%timeit sat.sgp4_tsince(tsince)

231 ns ± 7.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [75]:
# vectorise for one satellite over multiple times, many satellites over one time, many satellites over multiple times (do this later)...

def sgp4_many_times(n0, e0, i0, w0, Omega0, M0, t0, Bstar, tsince_array):
    """
    Vectorized SGP4 propagation over multiple times for a single satellite.
    
    Inputs:
      n0, e0, i0, w0, Omega0, M0, t0, Bstar : Orbital elements and parameters for the satellite
      tsince_array                        : Array of times since epoch (minutes)

    Returns:
    """
    jaxsgp4 = jax.jit(sgp4)
    sgp4_vectorized = jax.vmap(jaxsgp4, in_axes=(None, None, None, None, None, None, None, None, 0))
    return sgp4_vectorized(n0, e0, i0, w0, Omega0, M0, t0, Bstar, tsince_array)
    

In [78]:
# test propagation over multiple times

tsince_array = jnp.linspace(0, 1440, num=10000)  # 10 time points over one day
results_many_times = sgp4_many_times(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince_array)

%timeit sgp4_many_times(n0_test, e0_test, i0_test, w0_test, Omega0_test, M0_test, t0_test, Bstar_test, tsince_array).block_until_ready()

2.19 ms ± 21.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [79]:
# convert array of times to jd, fr for sgp4 package
jd, fr = sat.jdsatepoch, sat.jdsatepochF
jd_array = jd + np.array(tsince_array) / 1440.0
fr_array = fr * np.ones_like(tsince_array)

# check results against sgp4 package
_, true_r_many_times, true_v_many_times = sat.sgp4_array(jd_array, fr_array)
print(true_r_many_times - results_many_times[:, :3])  # compare positions
print(true_v_many_times - results_many_times[:, 3:6])  # compare velocities

%timeit sat.sgp4_array(jd_array, fr_array)  # SGP4 C++ implementation for multiple times

[[ 5.00222086e-12  0.00000000e+00 -1.18546514e-12]
 [ 2.75313946e-05  1.25707866e-05  3.82526780e-05]
 [ 5.46753263e-05  2.60099077e-05  7.64944377e-05]
 ...
 [ 5.66430913e-05  2.13257745e-05  7.64600437e-05]
 [ 2.81623929e-05  1.11103855e-05  3.82194978e-05]
 [-7.27595761e-11 -3.00133252e-11 -9.22568688e-11]]
[[-8.88178420e-16  1.77635684e-15  8.88178420e-16]
 [-2.22695444e-08  5.03146462e-08 -4.21648494e-10]
 [-4.51437598e-08  1.00351676e-07 -1.68662240e-09]
 ...
 [-3.65384549e-08  1.03693559e-07 -2.02996464e-09]
 [-1.85786115e-08  5.17253844e-08 -1.43578216e-09]
 [ 4.79616347e-14 -1.28341782e-13  4.44089210e-15]]
1.35 ms ± 8.45 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
# def sgp4_many_sats():

# def sgp4_grid():